In [1]:
from BayesNet import BayesNet
from BNReasoner import BNReasoner
from BNGenerator import generate_DAG
import networkx as nx
import pandas as pd
import copy
import random

In [2]:
#TD: If at some point you get a "omp Error #15," try running this code. It worked for me, but apparently it is not the best solution as it can cause problems, so use with "caution" I suppose 

# import os 
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
def get_bn_reasoner(nlevels: int, nodesplevel: int):
    generate_DAG('BN.BIFXML', nlevels, nodesplevel)
    net = BayesNet()
    net.load_from_bifxml('BN.BIFXML')
    numedges = len(net.get_all_edges())
    variables = net.get_all_variables()
    return BNReasoner(net), numedges, variables

def get_random_vars(variables, numevidence, nummapvars):
    evvars = random.sample(variables, numevidence)
    evidence = {}
    for var in evvars:
        evidence[var] = True
    
    mapvars = []
    i = 0
    while i < nummapvars:
        mapvar = random.sample(variables, 1)
        if mapvar not in evvars:
            mapvars.append(mapvar[0])
            i += 1
            
    if numevidence == 0:
        return None, mapvars
            
    return pd.Series(evidence), mapvars
    
heuristics = ['random', 'minfill', 'mindegree']

In [4]:
(1.5*20*3*2*2)/60

6.0

# Testrun

In [5]:
#test mpe 
results = pd.DataFrame()
for dimension in range(3, 5): # 7 in real run 
    print(dimension)
    numnodes = dimension**2
    for numevidence in range(0, (numnodes-1), int(numnodes//2)):
        for heuristic in heuristics:
            for i in range(3): # change this to 30? in real run 
                nummapvars = 0
                r, numedges, variables = get_bn_reasoner(dimension, dimension)
                evidence, mapvars = get_random_vars(variables, numevidence, nummapvars)
                try:
                    if type(evidence) != pd.Series:
                        instantiation, rowsmult, rowssum, rowsmax, timeelapsed = r.MPE(heuristic)
                    else:
                        instantiation, rowsmult, rowssum, rowsmax, timeelapsed = r.MPE(heuristic, evidence)
                    res = pd.DataFrame({'Memory Error': 'No',
                                            'Number Edges': [numedges],
                                            'Number Nodes': [numnodes],
                                            'Number Evidence': [numevidence],
                                            'Number MAP variables: ': [nummapvars],
                                            'Heuristic': heuristic,
                                            'Instantiation': [instantiation],
                                            'rows multiplied': [rowsmult],
                                            'rows summed out': [rowssum],
                                            'rows maxed out': [rowsmax],
                                            'seconds elapsed': [timeelapsed]
                                           })
                except:
                    res = pd.DataFrame({'Memory Error': 'Yes',
                                        'Number Edges': [numedges],
                                        'Number Nodes': [numnodes],
                                        'Number Evidence': [numevidence],
                                        'Number MAP variables: ': [nummapvars],
                                        'Heuristic': heuristic
                                       })
                results = results.append(res)
                results.to_csv('results_test.csv') # change filename !!!!

print('Is it a neat dataframe? Hakuna Matata!')
results

3
4
Is it a neat dataframe? Hakuna Matata!


,Memory Error,Number Edges,Number Nodes,Number Evidence,Number MAP variables:,Heuristic,Instantiation,rows multiplied,rows summed out,rows maxed out,seconds elapsed
0,No,6,9,0,0,random,l0n0 l2n1 l1n2 l2n2 l0n2 l1n0 l2n...,36.0,0.0,50.0,0.488870
0,No,6,9,0,0,random,l2n2 l1n2 l0n2 l1n0 l2n1 l1n1 l2...,42.0,0.0,56.0,0.720061
0,No,6,9,0,0,random,l1n2 l0n1 l2n0 l0n0 l1n0 l2n1 l0n2...,38.0,0.0,49.0,0.543519
0,No,6,9,0,0,minfill,l2n0 l2n2 l0n0 l0n2 l0n1 l1n0 l1...,45.0,0.0,53.0,0.668575
0,No,6,9,0,0,minfill,l2n1 l2n0 l0n0 l0n2 l1n1 l0n1 l1n0...,32.0,0.0,41.0,0.450553
0,Yes,6,9,0,0,minfill,NaN,NaN,NaN,NaN,NaN
0,No,6,9,0,0,mindegree,l2n1 l2n2 l0n0 l0n1 l0n2 l1n0 l1n1 ...,56.0,0.0,68.0,0.375541
0,No,6,9,0,0,mindegree,l2n2 l0n2 l1n1 l2n1 l0n0 l0n1 l...,32.0,0.0,38.0,0.380233
0,No,6,9,0,0,mindegree,l2n0 l2n1 l0n0 l0n2 l1n1 l0n1 l1n0 ...,30.0,0.0,40.0,0.379526
0,No,6,9,4,0,random,l1n0 l2n1 l0n0 l2n2 l0n1 l1n2 l0n2...,10.0,0.0,22.0,0.316682


In [6]:
#testmap
results = pd.DataFrame()
for dimension in range(3, 5): # 7 in real run 
    print(dimension)
    numnodes = dimension**2
    for numevidence in range(0, (numnodes-1), numnodes//2):
        for nummapvars in range(1, (numnodes-1), numnodes//2):
            for heuristic in heuristics:
                for i in range(3): # change this to 30? in real run 
                    r, numedges, variables = get_bn_reasoner(dimension, dimension)
                    evidence, mapvars = get_random_vars(variables, numevidence, nummapvars)
                    try:
                        if type(evidence) != pd.Series:
                            instantiation, rowsmult, rowssum, rowsmax, timeelapsed = r.MAP(heuristic, mapvars)
                        else:
                            instantiation, rowsmult, rowssum, rowsmax, timeelapsed = r.MAP(heuristic, mapvars, evidence)
                        res = pd.DataFrame({'Memory Error': 'No',
                                            'Number Edges': [numedges],
                                            'Number Nodes': [numnodes],
                                            'Number Evidence': [numevidence],
                                            'Number MAP variables: ': [nummapvars],
                                            'Heuristic': heuristic,
                                            'Instantiation': [instantiation],
                                            'rows multiplied': [rowsmult],
                                            'rows summed out': [rowssum],
                                            'rows maxed out': [rowsmax],
                                            'seconds elapsed': [timeelapsed]
                                           })
                    except:
                        res = pd.DataFrame({'Memory Error': 'Yes',
                                            'Number Edges': [numedges],
                                            'Number Nodes': [numnodes],
                                            'Number Evidence': [numevidence],
                                            'Number MAP variables: ': [nummapvars],
                                            'Heuristic': heuristic
                                           })
                    results = results.append(res)
                    results.to_csv('results_testMAP.csv') # change filename !!!

print('Is it a neat dataframe? Hakuna Matata!')
results

3
4
Is it a neat dataframe? Hakuna Matata!


,Memory Error,Number Edges,Number Nodes,Number Evidence,Number MAP variables:,Heuristic,Instantiation,rows multiplied,rows summed out,rows maxed out,seconds elapsed
0,No,6,9,0,1,random,p l2n0 0 0.565746 True,72,72,2,0.164072
0,No,6,9,0,1,random,p l2n0 0 0.511511 False,60,60,2,0.107848
0,No,6,9,0,1,random,p l1n1 0 0.784 False,4,4,2,0.030000
0,No,6,9,0,1,minfill,p l0n0 0 0.71 True,0,0,2,0.012636
0,No,6,9,0,1,minfill,p l1n0 0 0.98 False,0,0,2,0.014053
...,...,...,...,...,...,...,...,...,...,...,...
0,No,12,16,8,9,minfill,l3n0 l1n3 l1n2 l0n3 l0n2 l3n3 l2n3 ...,24,24,20,0.880239
0,No,12,16,8,9,minfill,l3n3 l1n3 l0n1 l1n1 l3n1 l0n0 ...,24,22,16,0.522754
0,No,12,16,8,9,mindegree,l2n2 l0n2 l2n1 l2n3 l3n2 l0n1 ...,8,14,12,0.388742
0,No,12,16,8,9,mindegree,l0n0 l0n2 l1n3 l2n0 l1n1 l3n3 l3n...,18,16,16,0.580504


In [ ]:
#estimate time
r, edges, variables = get_bn_reasoner(6, 6)
instantiation, rowsmult, rowssum, rowsmax, timeelapsed = r.MPE('minfill')
print('The next cell will probably run in about ' + str(((timeelapsed*30*3*2*2)/60)/60)) + ' hours. Good Luck, ' + 
    'dont forget to plug in your laptop!'

# MPE

## Attention: If you dont want to lose data, dont run this twoce without changing the filename

In [ ]:
#test mpe 
results = pd.DataFrame()
for dimension in range(3, 7): # 7 in real run 
    print(dimension)
    numnodes = dimension**2
    for numevidence in range(0, (numnodes-1), int(numnodes//2)):
        for heuristic in heuristics:
            for i in range(30): # change this to 30? in real run 
                nummapvars = 0
                r, numedges, variables = get_bn_reasoner(dimension, dimension)
                evidence, mapvars = get_random_vars(variables, numevidence, nummapvars)
                try:
                    if type(evidence) != pd.Series:
                        instantiation, rowsmult, rowssum, rowsmax, timeelapsed = r.MPE(heuristic)
                    else:
                        instantiation, rowsmult, rowssum, rowsmax, timeelapsed = r.MPE(heuristic, evidence)
                    res = pd.DataFrame({'Memory Error': 'No',
                                            'Number Edges': [numedges],
                                            'Number Nodes': [numnodes],
                                            'Number Evidence': [numevidence],
                                            'Number MAP variables: ': [nummapvars],
                                            'Heuristic': heuristic,
                                            'Instantiation': [instantiation],
                                            'rows multiplied': [rowsmult],
                                            'rows summed out': [rowssum],
                                            'rows maxed out': [rowsmax],
                                            'seconds elapsed': [timeelapsed]
                                           })
                except:
                    res = pd.DataFrame({'Memory Error': 'Yes',
                                        'Number Edges': [numedges],
                                        'Number Nodes': [numnodes],
                                        'Number Evidence': [numevidence],
                                        'Number MAP variables: ': [nummapvars],
                                        'Heuristic': heuristic
                                       })
                results = results.append(res)
                results.to_csv('results_MPE.csv') # change filename !!!!

print('Is it a neat dataframe? Hakuna Matata!')
results

3
4


# MAP

## Attention: If you dont want to lose data, dont run this twoce without changing the filename

In [ ]:
#testmap
results = pd.DataFrame()
for dimension in range(3, 7): # 7 in real run 
    print(dimension)
    numnodes = dimension**2
    for numevidence in range(0, (numnodes-1), numnodes//2):
        for nummapvars in range(1, (numnodes-1), numnodes//2):
            for heuristic in heuristics:
                for i in range(30): # change this to 30? in real run 
                    r, numedges, variables = get_bn_reasoner(dimension, dimension)
                    evidence, mapvars = get_random_vars(variables, numevidence, nummapvars)
                    try:
                        if type(evidence) != pd.Series:
                            instantiation, rowsmult, rowssum, rowsmax, timeelapsed = r.MAP(heuristic, mapvars)
                        else:
                            instantiation, rowsmult, rowssum, rowsmax, timeelapsed = r.MAP(heuristic, mapvars, evidence)
                        res = pd.DataFrame({'Memory Error': 'No',
                                            'Number Edges': [numedges],
                                            'Number Nodes': [numnodes],
                                            'Number Evidence': [numevidence],
                                            'Number MAP variables: ': [nummapvars],
                                            'Heuristic': heuristic,
                                            'Instantiation': [instantiation],
                                            'rows multiplied': [rowsmult],
                                            'rows summed out': [rowssum],
                                            'rows maxed out': [rowsmax],
                                            'seconds elapsed': [timeelapsed]
                                           })
                    except:
                        res = pd.DataFrame({'Memory Error': 'Yes',
                                            'Number Edges': [numedges],
                                            'Number Nodes': [numnodes],
                                            'Number Evidence': [numevidence],
                                            'Number MAP variables: ': [nummapvars],
                                            'Heuristic': heuristic
                                           })
                    results = results.append(res)
                    results.to_csv('results_MAP.csv') # change filename !!!

print('Is it a neat dataframe? Hakuna Matata!')
results